In [1]:
import torch
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_from_disk
from rouge_score import rouge_scorer
from bert_score import score

In [ ]:
# 로컬 경로 설정
model_path = "./model"     # 모델 경로
dataset_path = "./dataset" # 데이터셋 경로

# GPU 설정 (가능하면 CUDA 사용, 아니면 CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

# 모델과 토크나이저 로드 (GPU로 이동)
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)

# 데이터셋 로드 및 샘플링 (100개 샘플링)
dataset = load_from_disk(dataset_path)
test_data = dataset["test"].select(range(100)) # 상위 100개 샘플 선택

# ROUGE 및 BERTScore 준비
rouge_scorer_tool = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
rouge_results = {"rouge1": [], "rouge2": [], "rougeL": []}
references = []
hypotheses = []

# 배치 처리 설정
batch_size = 8 # 적절한 배치 크기 설정

In [ ]:
# 배치 단위로 요약 및 평가 진행
for i in tqdm(range(0, len(test_data), batch_size), desc="Evaluating"):
    batch = test_data.select(range(i, min(i + batch_size, len(test_data))))
    input_texts = batch["document"]
    reference_summaries = batch["summary"]

    # 텍스트를 토큰화하고 GPU로 전송
    inputs = tokenizer(input_texts, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    # 모델 요약 생성
    summary_ids = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    generated_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)

    # ROUGE 점수 계산
    for ref, gen in zip(reference_summaries, generated_summaries):
        scores = rouge_scorer_tool.score(ref, gen)
        for key in rouge_results:
            rouge_results[key].append(scores[key].fmeasure)

    # BERTScore를 위한 준비
    references.extend(reference_summaries)
    hypotheses.extend(generated_summaries)

In [ ]:
# BERTScore 계산
precision, recall, f1 = score(hypotheses, references, lang="ko", verbose=True)

# 평가 결과 출력
print("\nROUGE Scores:")
for key, values in rouge_results.items():
    print(f"{key}: {sum(values) / len(values):.4f}")

print("\nBERTScore:")
print(f"Precision: {precision.mean().item():.4f}")
print(f"Recall: {recall.mean().item():.4f}")
print(f"F1: {f1.mean().item():.4f}")